In [15]:
using Pkg
Pkg.add("XLSX")
Pkg.add("Oscar")
import XLSX
using Oscar

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


<font size="5">
In the first step the variables and the corresponding polynomial ring are defined. The coefficients of the polynomials can either be zero or one only (from the 2-elements field). 
</font>

In [16]:
R1 = ResidueRing(ZZ, 2);
vo= ["e", "h", "I", "a", "c", "p", "H", "d", "x"];
myR, (e, h, I, a, c, p, H, d, x) = PolynomialRing(R1, vo);
myvars=[e, h, I, a, c, p, H, d, x];

<font size="5">
Now your xlsx-table is loaded. The table consists of entries 0 and 1.
The array 'indices' is also loaded. It includes an own labeling of the objects in the table (if wanted). 
</font>

In [17]:
chartable=XLSX.readdata("../Data/Retirement_short.xlsx", "Tabelle1", "A3:I1526");
indices=1:1524;

<font size="5">
The analysis can be restricted to a subset of the characteristics. Select some of them.
</font>

In [18]:
varnum=9; choice=1:9

1:9

<font size="5">
According to your choice, the Boolean entries are transformed into an array of polynomials.
</font>

In [19]:
chartable=chartable[:,choice];
myvars=myvars[choice];
expr_list=prod(myvars+ myvars.^0 -chartable[1,:]); 
for i=2:size(chartable,1)
    expr_list=vcat(expr_list, prod(myvars+ myvars.^0 -chartable[i,:])); 
end

<font size="5">
For the weighting of the monomials, we do some statistics about the marginal distribution of characteristics
</font>

In [20]:
monweight=ones(Int,varnum);
for i=1:size(myvars,1)
    print(myvars[i]);
    print(" true=");
    print(sum(chartable[:,i]));
    print(" false=");
    print(size(chartable,1)-sum(chartable[:,i]));
    print("\n");
    monweight[i]=sum(chartable[:,i])*(size(chartable,1)-sum(chartable[:,i]));
end
monweight

e true=666 false=858
h true=66 false=1458
I true=255 false=1269
a true=411 false=1113
c true=240 false=1284
p true=312 false=1212
H true=727 false=797
d true=374 false=1150
x true=255 false=1269


9-element Vector{Int64}:
 571428
  96228
 323595
 457443
 308160
 378144
 579419
 430100
 323595

<font size="5">
The algorithm generates an ideal in the set of Boolean Polynomials. This ideal represents every combination of characteristics which is not to be found in your table. The Gröbner Basis of this ideal is computed.
</font>

In [21]:
uq_expr_list=unique(expr_list);
expr=sum(uq_expr_list)+1;
generator=myvars.^2+myvars;
Y, m = quo(myR, ideal(myR, vcat(generator, expr)));
GB = groebner_basis(ideal(myR, vcat(generator, expr)), ordering = wdeglex(gens(myR),monweight))

88-element Vector{nmod_mpoly}:
 h^2 + h
 c^2 + c
 x^2 + x
 I^2 + I
 p^2 + p
 d^2 + d
 a^2 + a
 h*I*c*x + h*c*x
 h*c*p*x + h*c*x
 h*I*p*x + h*c*p*x
 e^2 + e
 H^2 + H
 h*I*d*x + h*I*x + h*c*d*x + h*c*x
 ⋮
 e*I*c*p*d + e*a*c*p*d + I*c*p*d + I*c*d*x + a*c*p*d + c*p*d*x
 e*I*a*p*x + e*I*p*d*x + e*I*p*x + e*a*p*d*x + h*a*c*x + h*c*x + I*a*p*d*x + I*c*p*x + I*p*d*x + I*p*x + a*p*d*x + c*p*d*x
 e*c*p*H*x + I*a*c*p*x + a*c*p*x + c*p*H*d*x + c*p*H*x + c*p*d*x
 e*I*a*p*d + e*I*a*p + e*I*c*p*H + e*I*c*p*d + e*a*c*p*x + e*a*p*d*x + e*a*p*x + h*I*a + h*I*c + h*I*x + h*a*c*x + h*a*c + h*a*d*x + h*a*d + h*c*H + h*c*d*x + h*c*d + I*a*c*p*d + I*a*c*p + I*a*p*H*d + I*a*p*H + I*a*p*d*x + I*a*p*d + I*a*p*x + I*a*p + I*c*p*H + I*c*p*d + I*c*p*x + a*c*p*H*x + a*p*H*d*x + a*p*H*x + c*p*H*d*x
 e*a*c*d*x + e*c*H*d*x + e*c*d*x + h*c*d*x + h*c*x + I*c*p*x + a*c*H*d*x + c*p*H*d*x
 e*I*a*c*H + e*I*a*c + e*I*c*H*x + e*I*c*H + e*I*c + I*a*c*H*x + I*a*c*H + I*a*c*x + I*a*c + I*c*H*x + I*c*H + I*c
 e*h*c + e*I*c*p*H + 

<font size="5">
According to the "rules" included in the Gröbner Basis the simplification can be applied to all entries in the loaded table:
</font>

In [14]:
wrtnorm=0;
for i = 1:size(uq_expr_list,1)
    #res=m(uq_expr_list[i]); simplify!(res);
    res=normal_form(uq_expr_list[i],GB);
    #res=m(normal_form(uq_expr_list[i],GB)); simplify!(res);
    counter=0;
    for j = 1:size(expr_list,1)
        if (uq_expr_list[i]==expr_list[j])
            counter=counter+1;
            printstyled(indices[j], color=:red);
            print(" ");
        end
    end
    printstyled(" (#", color=:red);
    printstyled(counter, color=:red);
    printstyled(")", color=:red);
    print("\n");
    printstyled(factor(uq_expr_list[i]), color=:green);
    if (wrtnorm==1)
        printstyled("=\n", color=:green);
        printstyled(uq_expr_list[i], color=:green);
    end
    print("\n");
    printstyled(res, bold=:true);
    print("\n\n");
end

1 10 33 41 61 70 73 79 96 105 119 126 127 137 151 161 164 167 169 171 176 197 200 208 237 254 257 261 262 269 294 295 307 335 349 350 373 374 416 425 427 437 453 456 458 462 469 473 483 498 499 516 529 543 581 586 602 605 624 639 649 651 652 677 679 687 715 717 722 726 773 774 780 784 794 806 824 837 845 857 889 894 901 912 934 936 941 952 953 962 968 980 992 1017 1026 1058 1059 1073 1078 1083 1086 1100 1106 1108 1120 1131 1140 1144 1145 1157 1160 1163 1206 1214 1224 1234 1247 1249 1253 1262 1271 1284 1286 1288 1304 1307 1308 1340 1350 1354 1361 1367 1369 1394 1416 1443 1452 1455 1462 1470 1474 1483 1495 1498 1504  (#145)
1 * e * (x + 1) * (d + 1) * (H + 1) * (p + 1) * (c + 1) * (a + 1) * (I + 1) * (h + 1)
e*h*a*p*H*d + e*h*a*p*H*x + e*h*a*p*H + e*h*a*p*d + e*h*a*p*x + e*h*a*p + e*h*a*H*d + e*h*a*H*x + e*h*a*H + e*h*a*d + e*h*a*x + e*h*a + e*h*p*H*d + e*h*p*H*x + e*h*p*H + e*h*p*d + e*h*p*x + e*h*p + e*h*H*d + e*h*H*x + e*h*H + e*h*d + e*h*x + e*h + e*I*a*H*d + e*I*a*H + e*I*a*d + e*I*

6 50 145 165 234 297 332 334 451 504 535 540 552 636 719 955 1021 1295 1306 1323 1386 1430 1514  (#23)
1 * a * H * (x + 1) * (d + 1) * (p + 1) * (c + 1) * (I + 1) * (h + 1) * (e + 1)
e*h*a*p*H*d + e*h*a*p*H*x + e*h*a*p*H + e*h*a*H*d + e*h*a*H*x + e*h*a*H + e*I*a*c + e*I*a*p + e*I*a*H + e*I*c*p*H + e*I*c*H + e*I*c + e*I*H*x + e*a*c*p*H*d + e*a*c*p*H + e*a*c*H*d + e*a*c*H*x + e*a*c*H + e*a*p*H*d + e*a*p*H + e*a*p*d*x + e*a*p*x + e*a*H*d*x + e*a*H*d + e*a*H*x + e*a*H + h*I*c*H + h*I*c + h*I*p*d + h*I*d + h*I*x + h*a*c*p + h*a*c*H + h*a*p*H*d*x + h*a*p*H*x + h*a*p + h*a*H*d + h*a*H*x + h*a*H + h*a*d*x + h*a*d + h*c*H*x + I*a*c*p*H*d + I*a*c*p*H + I*a*c*p*d + I*a*c*p*x + I*a*c*p + I*a*c*d*x + I*a*c + I*a*p + I*a*H + I*c*p*H + I*c*H*d*x + I*c*H + I*c + I*H*x + a*c*p*H*d + a*c*p*H + a*c*p*d*x + a*c*p*x + a*c*H*d + a*c*H*x + a*c*H + a*p*H*d*x + a*p*H*d + a*p*H*x + a*p*H + a*H*d*x + a*H*d + a*H*x + a*H + c*p*H*d*x

7 120 560 713 791 1043  (#6)
1 * I * H * (x + 1) * (d + 1) * (p + 1) * (c + 1) *

26 271 285 356 433 711 757 775 809 917 960 993 1063 1107 1142 1174 1341 1398 1409 1500  (#20)
1 * e * p * H * d * (x + 1) * (c + 1) * (a + 1) * (I + 1) * (h + 1)
e*I*a*p*H + e*I*c*p*H + e*I*p*H*d + e*a*p*H*d + e*a*p*H*x + e*c*p*H*d + e*p*H*d*x + e*p*H*d + h*I*a + h*I*c*H + h*I*H*x + h*a*c*H + h*a*p*H + h*a*p + h*a*H*d*x + h*a*H*d + h*c*H + h*c*d + I*a*c*p*H + I*a*c*p*d*x + I*a*c*d*x + I*a*c*x + I*a*p*x + I*a*d*x + I*a*x + I*c*p*H*d + I*c*p*H + I*c*p*d*x + I*c*p*x + I*p*H*d*x + a*c*p*H*d + a*c*p*H*x + a*p*H*d*x + a*p*H*x

27 999  (#2)
1 * I * c * H * (x + 1) * (d + 1) * (p + 1) * (a + 1) * (h + 1) * (e + 1)
e*I*a*c + e*I*a*p + e*I*a*H*d + e*I*c*p*H + e*I*c*H*d + e*I*c + e*I*H*x + e*a*p*d*x + e*a*p*x + h*I*a + h*I*c + h*I*x + h*a*c + h*a*d*x + h*a*d + h*c*H + h*c*d + I*a*c*p*d + I*a*c*p + I*a*c*x + I*a*c + I*a*p*H*d + I*a*p*H + I*a*p*d*x + I*a*p*x + I*a*p + I*a*H*d + I*a*d*x + I*a*x + I*c*p*H + I*c*p*x + I*c*H*d + I*c + I*H*x + a*c*p*H*d*x + a*c*p*d*x + a*c*p*x + a*p*H*d*x + a*p*H*x + c*

45 296 513 891 942  (#5)
1 * e * p * H * x * (d + 1) * (c + 1) * (a + 1) * (I + 1) * (h + 1)
e*h*a*p*H*x + e*h*p*H*x + e*a*p*H*d*x + e*a*p*H*x + e*p*H*d*x + e*p*H*x + h*a*p*H*d*x + h*p*H*d*x + I*a*c*p*d*x + I*a*p*d*x + I*a*p*x + I*c*p*d*x + I*p*H*d*x + I*p*H*x + a*c*p*H*x + c*p*H*x

46 47 89 142 209 224 231 304 448 477 519 533 563 611 645 688 707 714 729 753 755 810 1137 1189 1212 1239 1244 1256 1322 1347 1358 1419 1475 1490 1501  (#35)
1 * e * H * (x + 1) * (d + 1) * (p + 1) * (c + 1) * (a + 1) * (I + 1) * (h + 1)
e*h*a*p*H*d + e*h*a*p*H*x + e*h*a*p*H + e*h*a*H*d + e*h*a*H*x + e*h*a*H + e*h*p*H*d + e*h*p*H*x + e*h*p*H + e*h*H*d + e*h*H*x + e*h*H + e*I*a*c + e*I*a*p + e*I*a*H + e*I*c*H*d + e*I*c + e*I*p*H*d + e*I*p*H + e*I*H*d + e*I*H*x + e*I*H + e*a*c*p*H + e*a*c*H*d + e*a*c*H*x + e*a*c*H + e*a*p*H*d + e*a*p*H + e*a*p*d*x + e*a*p*x + e*a*H*d*x + e*a*H*d + e*a*H*x + e*a*H + e*c*p*H*d + e*c*p*H + e*c*H*d*x + e*c*H*d + e*c*H*x + e*c*H + e*p*H*d*x + e*p*H*d + e*p*H*x + e*p*H + e*H*d*x + e

83 244 252 325 345 370 409 492 524 577 582 585 658 701 709 748 756 826 878 1018 1113 1129 1197 1222 1275 1314 1316 1326 1469 1515  (#30)
1 * d * (x + 1) * (H + 1) * (p + 1) * (c + 1) * (a + 1) * (I + 1) * (h + 1) * (e + 1)
e*h*a*p*H*d + e*h*a*p*d + e*h*a*H*d + e*h*a*d + e*h*p*H*d + e*h*p*d + e*h*H*d + e*h*d + e*I*a*p*H + e*I*a*p + e*I*a*H*d + e*I*a*d + e*I*p*H*d + e*I*p*d + e*I*H*d + e*I*H*x + e*I*d + e*I*x + e*a*c*H*d + e*a*c*d + e*a*p*H*d + e*a*p*H*x + e*a*p*d + e*a*p*x + e*a*H*d*x + e*a*H*d + e*a*d*x + e*a*d + e*c*p*H*d + e*c*p*d + e*c*H*d + e*c*d + e*p*H*d*x + e*p*H*d + e*p*d*x + e*p*d + e*H*d*x + e*H*d + e*d*x + e*d + h*I*c*H + h*I*c + h*I*H*x + h*I*x + h*a*c*H + h*a*c + h*a*p*H*d*x + h*a*p*H*d + h*a*p*H + h*a*p*d*x + h*a*p*d + h*a*p + h*p*H*d*x + h*p*H*d + h*p*d*x + h*p*d + h*H*d*x + h*H*d + h*d*x + h*d + I*a*c*p*H*d + I*a*c*p*H + I*a*c*p*d + I*a*c*p + I*a*p*H + I*a*p + I*a*H*d + I*a*d + I*c*H*d*x + I*c*d*x + I*p*H*d + I*p*d + I*H*d + I*H*x + I*d + I*x + a*c*p*H*x + a*c*p*x + a*c

143 357 438  (#3)
1 * e * p * d * x * (H + 1) * (c + 1) * (a + 1) * (I + 1) * (h + 1)
e*a*p*H*d*x + e*a*p*d*x + e*p*H*d*x + e*p*d*x + h*a*p*H*d*x + h*a*p*H*d + h*a*p*d*x + h*a*p*d + h*p*H*d*x + h*p*H*d + h*p*d*x + h*p*d + I*c*H*d*x + I*c*d*x + I*p*H*d*x + I*p*d*x

146 314 670 739 1273  (#5)
1 * p * d * (x + 1) * (H + 1) * (c + 1) * (a + 1) * (I + 1) * (h + 1) * (e + 1)
e*I*a*p*H + e*I*a*p + e*I*p*H*d + e*I*p*d + e*a*p*H*d + e*a*p*H*x + e*a*p*d + e*a*p*x + e*c*p*H*d + e*c*p*d + e*p*H*d*x + e*p*H*d + e*p*d*x + e*p*d + h*I*c*H + h*I*c + h*I*H*x + h*I*x + h*a*c*H + h*a*c + h*a*p*H*d + h*a*p*H + h*a*p*d + h*a*p + h*a*H*d*x + h*a*H*d + h*a*d*x + h*a*d + h*p*H*d + h*p*d + I*a*c*p*H*d + I*a*c*p*H + I*a*c*p*d + I*a*c*p + I*a*p*H + I*a*p + I*p*H*d + I*p*d + a*c*p*H*x + a*c*p*x + a*p*H*d*x + a*p*H*d + a*p*d*x + a*p*d + c*p*H*d + c*p*d + p*H*d*x + p*H*d + p*d*x + p*d

154 712 1233 1402  (#4)
1 * e * c * p * (x + 1) * (d + 1) * (H + 1) * (a + 1) * (I + 1) * (h + 1)
e*a*c*p*H*d + e*a*c*p*H + e*a*c*p

282 591  (#2)
1 * e * a * c * H * x * (d + 1) * (p + 1) * (I + 1) * (h + 1)
e*a*c*H*x + I*a*c*p*d*x + I*a*c*x + I*c*H*d*x + a*c*H*d*x + c*p*H*d*x

283 327 429 579 792 1363 1410  (#7)
1 * e * c * d * (x + 1) * (H + 1) * (p + 1) * (a + 1) * (I + 1) * (h + 1)
e*a*c*p*H*d + e*a*c*p*d + e*a*c*H*d + e*a*c*d + e*c*p*H*d + e*c*p*d + e*c*H*d + e*c*d + I*a*c*p*H*d + I*a*c*p*d + I*a*c*H*d + I*a*c*d + I*c*p*H*d + I*c*p*d + I*c*H*d*x + I*c*H*d + I*c*d*x + I*c*d

284 286 574 727 835 890 1088 1303 1311 1387  (#10)
1 * h * (x + 1) * (d + 1) * (H + 1) * (p + 1) * (c + 1) * (a + 1) * (I + 1) * (e + 1)
e*h*a*p*H*d + e*h*a*p*H*x + e*h*a*p*H + e*h*a*p*d + e*h*a*p*x + e*h*a*p + e*h*a*H*d + e*h*a*H*x + e*h*a*H + e*h*a*d + e*h*a*x + e*h*a + e*h*p*H*d + e*h*p*H*x + e*h*p*H + e*h*p*d + e*h*p*x + e*h*p + e*h*H*d + e*h*H*x + e*h*H + e*h*d + e*h*x + e*h + h*a*p*H*d*x + h*a*p*H*d + h*a*p*H*x + h*a*p*H + h*a*p*d*x + h*a*p*d + h*a*p*x + h*a*p + h*a*H*d*x + h*a*H*d + h*a*H*x + h*a*H + h*a*d*x + h*a*d + h*a*x + h*a + h

463 465  (#2)
1 * e * c * (x + 1) * (d + 1) * (H + 1) * (p + 1) * (a + 1) * (I + 1) * (h + 1)
e*a*c*p*H*d + e*a*c*p*H + e*a*c*p*d + e*a*c*p + e*a*c*H*d + e*a*c*H*x + e*a*c*H + e*a*c*d + e*a*c*x + e*a*c + e*c*p*H*d + e*c*p*H + e*c*p*d + e*c*p + e*c*H*d + e*c*H*x + e*c*H + e*c*d + e*c*x + e*c + I*a*c*p*H*d + I*a*c*p*H + I*a*c*p*d + I*a*c*p + I*a*c*H*d + I*a*c*H + I*a*c*d + I*a*c + I*c*p*H*d + I*c*p*H + I*c*p*d + I*c*p + I*c*H*d*x + I*c*H*d + I*c*H*x + I*c*H + I*c*d*x + I*c*d + I*c*x + I*c + a*c*p*H*d*x + a*c*p*d*x + c*p*H*d*x + c*p*d*x

467 617 745 839 973 1002  (#6)
1 * e * a * c * d * (x + 1) * (H + 1) * (p + 1) * (I + 1) * (h + 1)
e*I*c*H*d + e*I*c*d + e*a*c*H*d + e*a*c*d + e*c*H*d*x + e*c*d*x + I*a*c*p*H*d + I*a*c*p*d + I*a*c*H*d + I*a*c*d + I*c*H*d + I*c*d + a*c*p*H*d + a*c*p*d + c*p*H*d*x + c*p*d*x

480 1013 1305  (#3)
1 * e * a * p * H * d * (x + 1) * (c + 1) * (I + 1) * (h + 1)
e*I*a*p*H + e*I*c*p*H + e*a*c*p*H*d + e*a*p*H*d + e*a*p*H*x + h*I*a + h*I*c*H + h*I*H*x + h*a*c*H + h*a

797 1360  (#2)
1 * a * c * d * (x + 1) * (H + 1) * (p + 1) * (I + 1) * (h + 1) * (e + 1)
e*I*c*H*d + e*I*c*d + e*a*c*H*d + e*a*c*d + e*c*H*d*x + e*c*d*x + I*c*H*d + I*c*d + a*c*p*H*d*x + a*c*p*d*x + a*c*H*d*x + a*c*H*d + a*c*d*x + a*c*d + c*p*H*d*x + c*p*d*x

827  (#1)
1 * e * h * I * p * (x + 1) * (d + 1) * (H + 1) * (c + 1) * (a + 1)
e*h*a*p*H*d + e*h*a*p*H + e*h*a*p*d + e*h*a*p + e*h*p*H*d + e*h*p*H + e*h*p*d + e*h*p + h*I*H*d + h*I*H + h*I*d + h*I + h*a*p*H*d*x + h*a*p*d*x + h*p*H*d + h*p*H + h*p*d + h*p + I*c*H*d*x + I*c*d*x

829 1045 1372  (#3)
1 * I * H * d * (x + 1) * (p + 1) * (c + 1) * (a + 1) * (h + 1) * (e + 1)
e*I*a*p*H + e*I*a*p + e*I*c*H*d + e*I*p*H*d + e*I*H*d + e*I*H*x + e*a*c*p*H*d + e*a*p*H*d*x + e*a*p*H*x + e*a*p*d*x + e*a*p*x + h*I*c*H + h*I*c + h*I*H*x + h*I*x + h*a*c*p*d + h*a*c*H + h*a*c*d + h*a*c + h*a*p*H + h*a*p + h*a*H*d*x + h*a*H*d + h*a*d*x + h*a*d + h*c*p*d + h*c*d + I*a*c*p*H*d + I*a*c*p*H + I*a*c*p*d + I*a*c*p + I*a*p*H + I*a*p + I*c*H*d*x + I*c*H*d + I